In [1]:
import pytest
from matsciml.datasets import lips
from matsciml.datasets.lips import LiPSDataset, lips_devset
from matsciml.datasets import transforms

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dset = LiPSDataset(lips_devset)

In [3]:
sample = dset[10]

In [5]:
import pytorch_lightning as pl
from matsciml.models.pyg import EGNN
from matsciml.models.base import ScalarRegressionTask
from matsciml.lightning.data_utils import MatSciMLDataModule
from matsciml.datasets.transforms import DistancesTransform

In [15]:
# construct a scalar regression task with SchNet encoder
task = ScalarRegressionTask(
    encoder_class=EGNN,
    encoder_kwargs={"hidden_dim": 128, "output_dim": 64},
    task_keys=["energy"],
)

# task = ScalarRegressionTask(
#     encoder_class=EGNN,
#     # kwargs to be passed into the creation of SchNet model
#     encoder_kwargs={
#         "encoder_only": True,
#         "hidden_dim": 128,
#         "output__dim": 1,
#     },
#     # which keys to use as targets
#     task_keys=["energy"],
# )
# # Use IS2RE devset to test workflow

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [16]:
from matsciml.datasets.transforms import PointCloudToGraphTransform

# SchNet uses RBFs, and expects edge features corresponding to atom-atom distances
dm = MatSciMLDataModule.from_devset(
    "LiPSDataset",dset_kwargs={"transforms":[PointCloudToGraphTransform("pyg", cutoff_dist=15.0)]}
)

# run a quick training loop
trainer = pl.Trainer(fast_dev_run=1000)
trainer.fit(task, datamodule=dm)


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Running in `fast_dev_run` mode: will run the requested loop using 1000 batch(es). Logging and checkpointing is suppressed.
c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\pytorch_lightning\utilities\model_summary\model_summary.py:411: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(

  | Name         | Type       | Params
--------------------------------------------
0 | encoder      | EGNN       | 566 K 
1 | loss_func    | MSELoss    | 0     
2 | output_heads | ModuleDict | 0     
---

Epoch 0:  10%|█         | 5/50 [00:03<00:32,  1.38it/s, loss=1.28e+05, v_num=, train_energy=1.27e+5]

In [6]:
from matsciml.models.pyg.mace.modules.blocks import *
from matsciml.models.pyg.mace.modules.models import ScaleShiftMACE

In [7]:
import matsciml
import e3nn

In [8]:
model_config = dict(
        r_max=5.0,
        num_bessel=8,
        num_polynomial_cutoff=5,
        max_ell=3,
        interaction_cls= RealAgnosticResidualInteractionBlock ,
        num_interactions=2,
        num_elements=3,
        hidden_irreps=e3nn.o3.Irreps('16x0e+16x1o+16x2e'),
        atomic_energies=torch.Tensor([-13.663181292231226, -1029.2809654211628, -2042.0330099956639]),
        avg_num_neighbors=14.38,
        atomic_numbers=(1,6,8),
    )

In [27]:
model=ScaleShiftMACE(
            **model_config,
            correlation=3,
            gate=torch.nn.functional.silu,
            interaction_cls_first=RealAgnosticResidualInteractionBlock,
            MLP_irreps=e3nn.o3.Irreps('16x0e'),
            atomic_inter_scale=1.0,
            atomic_inter_shift=0.0,
        )

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\torch\jit\_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
f:\Vaibhav\M3RG\matsciml\matsciml\models\pyg\mace\modules\blocks.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_energies, dtype=torch.get_default_dtype()),


In [10]:
from matsciml.datasets.transforms import PointCloudToGraphTransform

# SchNet uses RBFs, and expects edge features corresponding to atom-atom distances
dm = MatSciMLDataModule.from_devset(
    "LiPSDataset",dset_kwargs={"transforms":[PointCloudToGraphTransform("pyg", cutoff_dist=15.0)]}
)


In [13]:
dm.setup()

In [16]:
Train_loader=dm.train_dataloader()

In [18]:
dataset_iter = iter(Train_loader)

In [20]:
batch=next(dataset_iter)

In [26]:
batch['graph']

DataBatch(edge_index=[2, 26536], pos=[664, 3], atomic_numbers=[664], force=[664, 3], batch=[664], ptr=[9])

In [28]:
model(batch)

Here


TypeError: _forward() got an unexpected keyword argument 'graph'

In [31]:
for key in batch.keys():
    try:
        print(key,batch[key].shape)
    except:
        print(key,batch[key])


cell torch.Size([24, 3])
energy torch.Size([8, 1])
force torch.Size([664, 3])
pbc torch.Size([8, 3])
targets {'energy': tensor([[-357.5916],
        [-357.8454],
        [-357.2771],
        [-357.1832],
        [-357.8833],
        [-357.1904],
        [-357.8636],
        [-357.9084]]), 'force': tensor([[-0.0536, -0.0860,  0.4961],
        [-0.2818, -0.5977,  0.7042],
        [ 0.0154, -0.6760,  0.4201],
        ...,
        [-0.0953, -0.8434, -1.1508],
        [-0.4223, -0.3521, -0.2732],
        [ 3.8550, -0.2919, -0.8205]])}
dataset LiPSDataset
graph DataBatch(edge_index=[2, 26536], pos=[664, 3], atomic_numbers=[664], force=[664, 3], batch=[664], ptr=[9])
target_types {'regression': ['energy', 'force'], 'classification': []}


In [32]:
for key in batch['graph'].keys():
    try:
        print(key,batch['graph'][key].shape)
    except:
        print(key,batch['graph'][key])

ptr torch.Size([9])
force torch.Size([664, 3])
atomic_numbers torch.Size([664])
pos torch.Size([664, 3])
edge_index torch.Size([2, 26536])
batch torch.Size([664])


In [ ]:
python3 mdbenchgnn/models/mace/scripts/run_train.py   --name="MACE_model_lips"   --log_dir="output_dir_sl/mace/lips/logs"   --model_dir="output_dir_sl/mace/lips"   --results_dir="output_dir_sl/mace/lips/results"   --checkpoints_dir="output_dir_sl/mace/lips/checkpoints"   --train_file='/home/civil/visitor/vaibhavb.visitor/ce1180169.home/MDBENCHGNN/example_mdbenchgnn/md17/aspirin/train/botnet.xyz'   --valid_file='/home/civil/visitor/vaibhavb.visitor/ce1180169.home/MDBENCHGNN/example_mdbenchgnn/md17/aspirin/val/botnet.xyz'   --E0s='{1:-13.663181292231226, 3:-216.78673811801755, 6:-1029.2809654211628, 7:-1484.1187695035828, 8:-2042.0330099956639, 15:-1537.0898574856286, 16:-1867.8202267974733}'   --model="ScaleShiftMACE"   --hidden_irreps='16x0e+16x1o+16x2e '   --r_max=5.0   --batch_size=20   --valid_batch_size=20   --max_num_epochs=500   --ema   --ema_decay=0.99   --amsgrad   --default_dtype="float32"   --device=cpu   --seed=123   --swa
